### Projeto Final de Spark - Avançado
#### Campanha Nacional de Vacinação contra Covid-19

Autor: Arthur Freitas Gomes

Turma: Semantix Academy - Big Data Engineer 04-21

Professor: Rodrigo Rebouças

##### Nível Avançado:

Replicar as visualizações do site “https://covid.saude.gov.br/”, porém acessando diretamente a API de Elastic.

Link oficial para todas as informações:  https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao

Informações para se conectar ao cluster:

    > URL https://imunizacao-es.saude.gov.br/desc-imunizacao
    > Nome do índice: desc-imunizacao
    > Credenciais de acesso
    > Usuário: imunizacao_public
    > Senha: qlto5t&7r_@+#Tlstigi

In [58]:
import requests
import pandas as pd
import time

In [59]:
#Acesso inicial para gravar '_scroll_id'
covid_API = requests.get('https://imunizacao-es.saude.gov.br/_search?scroll=1m',
                         auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                         params={"size":10000})

In [60]:
#covid_API.json()

In [61]:
#'_scroll_id' = id temporário que será utilizado para navegar nos scrolls com 10000 registros cada
covid_API.json()['_scroll_id']

'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoAxZNS1hZTVE5c1RFV1d5TnBQdV94RHVnAAAAAG15FTQWVGNTY2RHVmdRTUtxVjBiQXFFMEJCURZlR0trMDNRRlFreTlFRWtxel9vME1BAAAAAGR0XnoWeUtZa0E0b1ZTZlNjam5veUxfS1ZHdxYycGRZR2c4SVNXeWpFSkh6c0JjXzFBAAAAAG5vf5MWV3hBS3dIbGpTWkNDV0dBYWMwUUU0dw=='

In [62]:
#Número de registros por Scroll
len(covid_API.json()['hits']['hits'])

10000

In [63]:
#Scroll = 0 

count_scroll = 0

scroll_id = covid_API.json()['_scroll_id']
covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll',
                                auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                                params={"scroll_id":scroll_id, "scroll":'1m'})

#recebe infomações HITS, seleciona dict _source
df_covid = pd.DataFrame([x['_source'] for x in covid_API_scroll.json()['hits']['hits']])

In [66]:
%%time
#Loop para repetir processo anterior até hit  = 0

while count_scroll < 10:
#while len(covid_API_scroll.json()['hits']['hits']) != 0:
    scroll_id = covid_API.json()['_scroll_id']
    covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll', auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), params={"scroll_id":scroll_id, "scroll":'1m'})

    #covid_API_scroll.json()
    #covid_API_scroll.json()['hits']['hits']
    
    df_covid = df_covid.append([x['_source'] for x in covid_API_scroll.json()['hits']['hits']])
    count_scroll += 1
   

CPU times: user 7.16 s, sys: 323 ms, total: 7.48 s
Wall time: 11 s


In [67]:
%%time
covid_vac = spark.createDataFrame(df_covid)

In [68]:
covid_vac.count()

110000

##### Próximos passos:

    > conseguir dubir todos os hits;
    > criar visualizações.